# Bronze Layer Ingestion

This notebook ingests raw data from two sources and lands it in Bronze Delta tables.

- **1. Real-time Vehicle Updates**: A continuous stream from Azue Event Hubs.
- **2. Static GTFS Data**: A daily batch ZIP file from ADLS Gen2.

In [0]:
# Imports
from pyspark.sql import functions as F
import io
import zipfile
import os
import shutil
from pyspark.sql.functions import col, length, from_json, explode, from_unixtime
from pyspark.sql.types import StructType, StructField, MapType, StringType, IntegerType, DoubleType, LongType, DoubleType, ArrayType, TimestampType, BooleanType, FloatType


In [0]:
# Define widgets to accept parameters via DB job
dbutils.widgets.text("secret_scope_name", "delijn-secrets", "Secret Scope Name")
dbutils.widgets.text("eh_conn_str_key", "eh-conn-str", "Event Hub Connection String Key")

# Define file paths
SILVER_PATH = "/mnt/silver/"

REALTIME_TABLE_NAME = "realtime_trips"
BRONZE_REALTIME_CHECKPOINT = f"/mnt/bronze/checkpoints/{REALTIME_TABLE_NAME}"

BRONZE_GTFS_PATH = "/mnt/bronze/gtfs/gtfs_transit.zip"
UNZIPPED_GTFS_PATH = "/mnt/bronze/gtfs/gtfs_unzipped/"
TEMP_ZIP_PATH = "/tmp/gtfs_transit.zip"
TEMP_UNZIPPED_PATH = "/tmp/gtfs_unzipped/"


In [0]:
# Event body schema
time_schema = StructType([
    StructField("delay", LongType(), True),
    StructField("time", LongType(), True)
])

stop_time_update_schema = StructType([
    StructField("stopId", StringType(), True),
    StructField("arrival", time_schema, True),
    StructField("departure", time_schema, True),
    StructField("stopSequence", LongType(), True)
])

event_body_schema = StructType([
    StructField("id", StringType(), True),
    StructField("tripUpdate", StructType([
        StructField("trip", StructType([
            StructField("tripId", StringType(), True),
            StructField("startDate", StringType(), True)
        ]), True),
        StructField("stopTimeUpdate", ArrayType(stop_time_update_schema), True),
        StructField("vehicle", StructType([
             StructField("id", StringType(), True)
        ]), True),
        StructField("timestamp", LongType(), True)
    ]), True),
    StructField("vehicle", StructType([
        StructField("vehicle", StructType([
             StructField("id", StringType(), True)
        ]), True),
        StructField("trip", StructType([
            StructField("tripId", StringType(), True)
        ]), True),
        StructField("position", StructType([
            StructField("latitude", DoubleType(), True),
            StructField("longitude", DoubleType(), True)
        ]), True),
        StructField("timestamp", LongType(), True)
    ]), True)
])

## 1. Real-Time Data Ingestion (From Event Hubs)

In [0]:
# Load RT stream
scope = dbutils.widgets.get("secret_scope_name")
key = dbutils.widgets.get("eh_conn_str_key")
EH_CONN_STR = dbutils.secrets.get(scope=scope, key=key)

eh_conf = {
  'eventhubs.connectionString': sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(EH_CONN_STR),
  'eventhubs.consumerGroup': '$Default',
  'eventhubs.eventPosition': 'earliestEventPosition'
}

df_stream_raw = (
  spark.readStream
    .format("eventhubs")
    .options(**eh_conf)
    .load()
)

In [0]:
# Transform RT data
parsed_stream = (
    df_stream_raw
    .withColumn("body", col("body").cast(StringType()))
    .withColumn("event", from_json(col("body"), event_body_schema))
    .select(
        col("event.id").alias("update_id"),
        col("event.tripUpdate.trip.tripId").alias("trip_id"),
        col("event.tripUpdate.trip.startDate").alias("trip_start_date"),
        col("event.tripUpdate.vehicle.id").alias("vehicle_id"),
        col("event.vehicle.position.latitude").alias("latitude"),
        col("event.vehicle.position.longitude").alias("longitude"),
        from_unixtime(col("event.tripUpdate.timestamp")).cast(TimestampType()).alias("event_timestamp"),
        explode("event.tripUpdate.stopTimeUpdate").alias("stop_update") # Explode the array
    )
)

final_stream = parsed_stream.select(
    "update_id",
    "trip_id",
    "trip_start_date",
    "vehicle_id",
    "latitude",
    "longitude",
    "event_timestamp",
    col("stop_update.stopId").alias("stop_id"),
    col("stop_update.stopSequence").alias("stop_sequence"),
    col("stop_update.departure.delay").alias("departure_delay_seconds"),
    from_unixtime(col("stop_update.departure.time")).cast(TimestampType()).alias("departure_time")
)

In [0]:
# Write stream to silver layer
(
    final_stream.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", BRONZE_REALTIME_CHECKPOINT)
    .trigger(availableNow=True) # process all available data in a micro-batch; more efficient for scheduled jobs
    .start(f'{SILVER_PATH}/{REALTIME_TABLE_NAME}')
)

## 2. Static GTFS Batch Data Ingestion (From ADLS Gen2)

In [0]:
# Clean up
if os.path.exists(TEMP_ZIP_PATH):
    os.remove(TEMP_ZIP_PATH)

if os.path.exists(TEMP_UNZIPPED_PATH):
    shutil.rmtree(TEMP_UNZIPPED_PATH)

dbutils.fs.rm(UNZIPPED_GTFS_PATH, recurse=True)
dbutils.fs.mkdirs(UNZIPPED_GTFS_PATH)

True

In [0]:
# Copy ZIP to local
dbutils.fs.cp(BRONZE_GTFS_PATH, f"file:{TEMP_ZIP_PATH}")

try:
    with zipfile.ZipFile(TEMP_ZIP_PATH, 'r') as zip_ref:
        os.makedirs(TEMP_UNZIPPED_PATH, exist_ok=True)
        zip_ref.extractall(TEMP_UNZIPPED_PATH)

    # Copy unzipped to DBFS
    for file_name in os.listdir(TEMP_UNZIPPED_PATH):
        source_path = f"file://{TEMP_UNZIPPED_PATH}{file_name}"
        target_path = f"{UNZIPPED_GTFS_PATH}{file_name}"
        dbutils.fs.cp(source_path, target_path)

except Exception as e:
    print(f"An error occurred during unzipping.")
    raise e

In [0]:
# Txt files to process
files_to_process = {
    "stops.txt": {
        "schema": StructType([
            StructField("stop_id", StringType(), True),
            StructField("stop_code", StringType(), True),
            StructField("stop_name", StringType(), True),
            StructField("stop_lat", DoubleType(), True),
            StructField("stop_lon", DoubleType(), True),
            StructField("stop_url", StringType(), True),
            StructField("wheelchair_boarding", BooleanType(), True)
        ]),
        "silver_table_name": "stops",
        "transform_func": lambda df: df.select(
            col("stop_id"), col("stop_name"), col("stop_code"),
            col("stop_lat").alias("latitude"), col("stop_lon").alias("longitude"),
            col("wheelchair_boarding")
        )
    },
    "routes.txt": {
        "schema": StructType([
            StructField("route_id", StringType(), True),
            StructField("agency_id", StringType(), True),
            StructField("route_short_name", StringType(), True),
            StructField("route_long_name", StringType(), True),
            StructField("route_desc", StringType(), True),
            StructField("route_type", IntegerType(), True),
            StructField("route_url", StringType(), True),
            StructField("route_color", StringType(), True),
            StructField("route_text_color", StringType(), True)
        ]),
        "silver_table_name": "routes",
        "transform_func": lambda df: df  # Keep as is
    },
    "trips.txt": {
        "schema": StructType([
            StructField("route_id", StringType(), True),
            StructField("service_id", StringType(), True),
            StructField("trip_id", StringType(), True),
            StructField("trip_headsign", StringType(), True),
            StructField("trip_short_name", StringType(), True),
            StructField("direction_id", IntegerType(), True),
            StructField("block_id", StringType(), True),
            StructField("shape_id", StringType(), True)
        ]),
        "silver_table_name": "trips",
        "transform_func": lambda df: df
    },
    "stop_times.txt": {
        "schema": StructType([
            StructField("trip_id", StringType(), True),
            StructField("arrival_time", StringType(), True),
            StructField("departure_time", StringType(), True),
            StructField("stop_id", StringType(), True),
            StructField("stop_sequence", IntegerType(), True),
            StructField("pickup_type", IntegerType(), True),
            StructField("drop_off_type", IntegerType(), True)
        ]),
        "silver_table_name": "stop_times",
        "transform_func": lambda df: df
    },
    "calendar_dates.txt": {
        "schema": StructType([
            StructField("service_id", StringType(), True),
            StructField("date", IntegerType(), True), # YYYYMMDD
            StructField("exception_type", IntegerType(), True)
        ]),
        "silver_table_name": "calendar_dates",
        "transform_func": lambda df: df
    },
    "shapes.txt": {
        "schema": StructType([
            StructField("shape_id", StringType(), True),
            StructField("shape_pt_lat", DoubleType(), True),
            StructField("shape_pt_lon", DoubleType(), True),
            StructField("shape_pt_sequence", IntegerType(), True),
            StructField("shape_dist_traveled", FloatType(), True)
        ]),
        "silver_table_name": "shapes",
        "transform_func": lambda df: df
    }
}


In [0]:
# Load & process each file
for file_name, config in files_to_process.items():
    raw_path = f"{UNZIPPED_GTFS_PATH}/{file_name}"
    silver_path = f"{SILVER_PATH}/{config['silver_table_name']}"

    raw_df = (
        spark.read
        .format("csv")
        .option("header", "true")
        .schema(config['schema'])
        .load(raw_path)
    )

    transformed_df = config['transform_func'](raw_df)

    (
        transformed_df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(silver_path)
    )
    
    print(f"Successfully created silver table: {config['silver_table_name']}")

print("\n--- All static files processed successfully! ---")

Successfully created silver table: stops
Successfully created silver table: routes
Successfully created silver table: trips
Successfully created silver table: stop_times
Successfully created silver table: calendar_dates
Successfully created silver table: shapes

--- All static files processed successfully! ---
